In [127]:
import json
import csv
import pandas as pd
import gurobipy as gp
from datetime import datetime, timedelta

In [128]:
OBP_AVG = 0.327
SLG_AVG = 0.365

FPCT_AVG = {
    '2': 0.992,
    '3': 0.993,
    '4': 0.977, 
    '5': 0.929,
    '6': 0.964,
    '7': 0.977,
    '8': 0.988,
    '9': 0.976
}

POSITION_WEIGHT = {
    '2': {"batting": 0.95, "fielding": 1.05},
    '3': {"batting": 1.21, "fielding": 0.83},
    '4': {"batting": 0.97, "fielding": 1.03},
    '5': {"batting": 0.96, "fielding": 1.04},
    '6': {"batting": 0.94, "fielding": 1.06},
    '7': {"batting": 1.02, "fielding": 0.98},
    '8': {"batting": 1.04, "fielding": 0.96},
    '9': {"batting": 1.17, "fielding": 0.85},
}

OPPONENT_WEIGHT = {
    "中信兄弟": 1.15,
    "統一7-ELEVEn獅": 1.11,
    "樂天桃猿": 0.96, 
    "富邦悍將": 0.93,
    "味全龍": 0.85
}

dataRoot = "../clean/clean_all_"
playerRoot = "../球員異動/"
scheduleRoot = "../schedule/"

teamNameMap = {
    "中信兄弟": "brothers", 
    "味全龍": "dragons",
    "富邦悍將": "guardians", 
    "統一7-ELEVEn獅": "lions",
    "樂天桃猿": "monkeys"
}

fieldMap = {
    'DH': 'DH',
    '2': 'C',
    '3': '1B',
    '4': '2B',
    '5': '3B',
    '6': 'SS',
    '7': 'LF',
    '8': 'CF',
    '9': 'RF'
}

### Import Data

In [129]:
class Player:
    def __init__(self, name, data):
        self.name = name
        self.data = data

In [130]:
allData = {}
for team in teamNameMap.keys():
    with open(dataRoot + team + ".json", encoding="utf-8") as f:
        temp = []
        for line in f:
            player = json.loads(line)
            temp.append(Player(player["name"], player["data"]))
        allData[teamNameMap[team]] = temp

brothers = allData["brothers"]
dragons = allData["dragons"]
guardians = allData["guardians"]
lions = allData["lions"]
monkeys = allData["monkeys"]

In [131]:
playerList = {}

for team in teamNameMap.keys():
    tempDict = pd.read_csv(playerRoot + team + ".csv", header=0, index_col=0).to_dict()
    tempDict = {
        datetime.strptime(key, "%Y-%m-%d").date(): list(filter(lambda d: pd.isna(d) == False, list(value.values())))
        for key, value in tempDict.items()
    }
    playerList[teamNameMap[team]] = tempDict

In [132]:
def createDate(mm, dd):
    return datetime(2022, mm, dd, 0, 0, 0).date()

### Calculate Parameters

In [133]:
# 比賽當天 27 人名單去除投手
def findPlayerList(team, date):
    minDelta = timedelta(days=100)
    playerDay = date
    for key in playerList[teamNameMap[team]].keys():
        if timedelta(days=0) <= date - key < minDelta:
            minDelta = date - key
            playerDay = key
    
    withoutPitcherList = []
    for data in allData[teamNameMap[team]]:
        withoutPitcherList.append(data.name)
    return list(filter(lambda d: d in withoutPitcherList, playerList[teamNameMap[team]][playerDay]))
        

In [134]:
# A_{ij}
def calcA(player, pos):
    if str(pos) in player.data["fielding"]["pos"].keys():
        return 1
    else:
        return 0

In [135]:
# F_{ij}
def calcF(player, pos):
    if str(pos) in player.data["fielding"]["pos"].keys():
        F = float(player.data["fielding"]["pos"][str(pos)]["FPCT"]) / FPCT_AVG[str(pos)]
        return F
    else:
        return 0

In [136]:
# B_i
def calcOPS(OBP, SLG):
    return OBP / OBP_AVG + SLG / SLG_AVG - 1

def calcB(game, player, aMonth):
    if game["date"].month == 4:
        aMonth = "Apr"
    if game["date"].month == 5:
        aMonth = "May"
    if game["date"].month == 6:
        aMonth = "Jun"
    aOppo = game["oppo"]
    aPitcher = game["pitcher"]
    aField = game["field"]

    OPSseason = float(player.data["batting"]["season"]["OPS+"])
    PAseason = int(player.data["batting"]["season"]["PA"])
    
    if "month" in player.data["batting"].keys():
        if aMonth in player.data["batting"]["month"].keys():
            if int(player.data["batting"]["month"][aMonth]["AB"]) != 0:
                OPSmonth = calcOPS(
                    float(player.data["batting"]["month"][aMonth]["OBP"]), 
                    float(player.data["batting"]["month"][aMonth]["TB"]) / float(player.data["batting"]["month"][aMonth]["AB"])
                )
                PAmonth = int(player.data["batting"]["month"][aMonth]["PA"])
            else:
                OPSmonth = OPSseason
                PAmonth = 0
        else:
            OPSmonth = OPSseason
            PAmonth = 0
    else:
        OPSmonth = OPSseason
        PAmonth = 0

    if "field" in player.data["batting"]:
        if aField in player.data["batting"]["field"].keys():
            if int(player.data["batting"]["field"][aField]["AB"]) != 0:
                OPSfield = calcOPS(
                    float(player.data["batting"]["field"][aField]["OBP"]), 
                    float(player.data["batting"]["field"][aField]["TB"]) / float(player.data["batting"]["field"][aField]["AB"])
                )
                PAfield = int(player.data["batting"]["field"][aField]["PA"])
            else:
                OPSfield = OPSseason
                PAfield = 0
        else:
            OPSfield = OPSseason
            PAfield = 0
    else:
        OPSfield = OPSseason
        PAfield = 0
    
    if "vsP" in player.data["batting"].keys():
        if aOppo in player.data["batting"]["vsP"]["data"].keys():
            if aPitcher in player.data["batting"]["vsP"]["data"][aOppo].keys():
                if int(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["AB"]) != 0:
                    OPSvsp = calcOPS(
                        float(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["OBP"]), 
                        float(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["TB"]) / float(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["AB"])
                    )
                    PAvsp = int(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]["PA"])
                else:
                    OPSvsp = OPSseason
                    PAvsp = 0
            else:
                OPSvsp = OPSseason
                PAvsp = 0
        else:
            OPSvsp = OPSseason
            PAvsp = 0
    else:
        OPSvsp = OPSseason
        PAvsp = 0

    if PAseason + PAmonth + PAfield + PAvsp == 0:
        B = 0
    else:
        B = (OPSseason * PAseason + OPSmonth * PAmonth + OPSfield * PAfield + OPSvsp * PAvsp) / (PAseason + PAmonth + PAfield + PAvsp)
    return B

# V_{ij}
def calcV(game, month, player, pos):
    A = calcA(player, pos)
    B = calcB(game, player, month)
    F = calcF(player, pos)
    return (POSITION_WEIGHT[str(pos)]["batting"] * B + POSITION_WEIGHT[str(pos)]["fielding"] * F) * A

### Integer Program

#### Stage 1

In [137]:
def stage1(aTeam, aGame, aMonth):
    model = gp.Model("model")

    # decision variables
    x = []
    for g in range(len(aGame)):
        todayPlayerList = findPlayerList(aTeam, aGame[g]["date"])
        singleGame = {}
        for i in range(len(todayPlayerList)):
            singlePlayer = []
            for j in range(2, 10):
                singlePlayer.append(model.addVar(lb=0, vtype="B", name="x" + str(g) + "-" + todayPlayerList[i] + "-" + str(j)))
            singleGame[todayPlayerList[i]] = singlePlayer
        x.append(singleGame)

    # objective function
    model.setObjective(gp.quicksum(
        gp.quicksum(
            gp.quicksum(
                OPPONENT_WEIGHT[aGame[g]["oppo"]] * calcV(aGame[g], aMonth, list(filter(lambda d: d.name == thisPlayer, allData[teamNameMap[aTeam]]))[0], j) * x[g][thisPlayer][j - 2]
                for j in range(2, 10)
            ) for thisPlayer in findPlayerList(aTeam, aGame[g]["date"])
        ) for g in range(len(aGame))
    ), gp.GRB.MAXIMIZE)

    # constraints
    zSum = 0
    for g in range(len(aGame)):
        todayPlayerList = findPlayerList(aTeam, aGame[g]["date"])
        zSum += OPPONENT_WEIGHT[aGame[g]["oppo"]]
        for thisPlayerName in todayPlayerList:
        
            thisPlayer = list(filter(lambda d: d.name == thisPlayerName, allData[teamNameMap[aTeam]]))[0]
                    
            # constraint 2
            model.addConstr(gp.quicksum(x[g][thisPlayerName][j - 2] for j in range(2, 10)) <= 1)

            for j in range(2, 10):
                # constraint 3
                model.addConstr(x[g][thisPlayerName][j - 2] <= calcA(thisPlayer, j))

        for j in range(2, 10):
            # constraint 1
            model.addConstr(gp.quicksum(x[g][thisPlayerName][j - 2] for thisPlayerName in todayPlayerList) <= 1)

    # constraint 4
    for thisPlayer in findPlayerList(aTeam, aGame[0]["date"]):
        if len(aGame) >= 4:
            for n in range(4, len(aGame) + 1):
                model.addConstr(
                    gp.quicksum(
                        x[g][thisPlayer][2 - 2] if thisPlayer in findPlayerList(aTeam, aGame[g]["date"]) else 0
                        for g in range(n - 4, n)
                    ) <= 3
                )

    model.optimize()

    playerName = []
    onPlayer = {}
    for g in range(len(aGame)):
        singleGame = []
        onPlayerTemp = {}
        for j in range(8):
            for name in x[g].keys():
                if x[g][name][j].x == 1:
                    singleGame.append(name)
                    onPlayerTemp[name] = { "field": str(j + 2) }
        playerName.append(singleGame)
        onPlayer[aGame[g]["date"]] = onPlayerTemp
        print("===== complete game", g, "- stage 1 =====")
    
    return playerName, onPlayer, model.objVal, model.objVal / zSum

#### Stage 1.5 - DH

In [138]:
def fillField(aTeam, game, playerName, positionType):
    candidate = list(filter(lambda d: d not in playerName, findPlayerList(aTeam, game["date"])))
    print("***** candidate:", candidate)
    findPlayerList(aTeam, game["date"])
    # 內野手
    if positionType == 1:
        for can in candidate:
            thisPlayerObject = list(filter(lambda d: d.name == can, allData[teamNameMap[aTeam]]))[0]
            for pos in ["3", "4", "5", "6"]:
                if pos in thisPlayerObject.data["fielding"]["pos"].keys():
                    return can
    # 外野手
    else:
        for can in candidate:
            thisPlayerObject = list(filter(lambda d: d.name == can, allData[teamNameMap[aTeam]]))[0]
            for pos in ["7", "8", "9"]:
                if pos in thisPlayerObject.data["fielding"]["pos"].keys():
                    return can

In [139]:
def stageDH(aTeam, aGame, aMonth, playerName, onPlayer):
    for g in range(len(aGame)):
        if len(playerName[g]) < 8:
            onPositionRecord = [False, False, False, False, False, False, False, False]
            for player in onPlayer[aGame[g]["date"]].keys():
                onPositionRecord[int(onPlayer[aGame[g]["date"]][player]["field"]) - 2] = True
            for i in range(len(onPositionRecord)):
                if(onPositionRecord[i] == False):
                    if i <= 4:
                        fill = fillField(aTeam, aGame[g], playerName[g], 1)
                    else:
                        fill = fillField(aTeam, aGame[g], playerName[g], 2)
                    onPlayer[aGame[g]["date"]][fill] = { "field": str(i + 2) }
                    playerName[g].append(fill)
                    onPositionRecord[i] = True

        maxB = -1
        todayPlayerList = findPlayerList(aTeam, aGame[g]["date"])
        for todayPlayer in todayPlayerList:
            if todayPlayer not in playerName[g]:
                thisPlayerObject = list(filter(lambda d: d.name == todayPlayer, allData[teamNameMap[aTeam]]))[0]
                thisB = calcB(aGame[g], thisPlayerObject, aMonth)
                if thisB > maxB:
                    DHPlayerName = thisPlayerObject.name
                    maxB = thisB
        playerName[g].append(DHPlayerName)
        onPlayer[aGame[g]["date"]][DHPlayerName] = { "field": "DH" }

        print("===== complete game", g, "- stage DH =====")
    return onPlayer, playerName

#### Stage 2

In [140]:
def stage2(aTeam, aGame, onPlayer, playerName):
    stage2Obj = []
    for g in range(len(aGame)):
        model2 = gp.Model("model2")
        
        allO = []
        y = []
        for j in range(1, 10):
            thisPlayer = list(filter(lambda d: d.name == playerName[g][j - 1], allData[teamNameMap[aTeam]]))[0]
            temp = []
            tempY = []
            for k in range(1, 10):
                if "baorder" in thisPlayer.data["batting"].keys():
                    if str(k) in thisPlayer.data["batting"]["baorder"].keys():
                        thisOrderData = thisPlayer.data["batting"]["baorder"][str(k)]
                        PA = int(thisOrderData["PA"])
                        if PA >= 30:
                            O = (int(thisOrderData["IBB"][1]) * 0.72 + int(thisOrderData["HBP"]) * 0.75 
                            + (int(thisOrderData["H"]) - int(thisOrderData["2B"]) - int(thisOrderData["3B"]) - int(thisOrderData["HR"])) * 0.9
                            + int(thisOrderData["2B"]) * 1.24 + int(thisOrderData["3B"]) * 1.56 + int(thisOrderData["HR"]) * 1.95) / PA
                        else:
                            O = thisPlayer.data["batting"]["season"]["wOBA"]
                    else:
                        O = thisPlayer.data["batting"]["season"]["wOBA"]
                else:
                    O = thisPlayer.data["batting"]["season"]["wOBA"]
                temp.append(O)
                tempY.append(model2.addVar(lb=0, vtype="B", name="y" + str(j) + "-" + str(k)))

            allO.append(temp)
            y.append(tempY)
        
        model2.setObjective(
            gp.quicksum(
                gp.quicksum(
                    allO[j - 1][k - 1] * y[j - 1][k - 1] 
                    for k in range(1, 10)
                ) 
                for j in range(1, 10)
            ),
            gp.GRB.MAXIMIZE
        )

        for k in range(1, 10):
            model2.addConstr(gp.quicksum(y[j - 1][k - 1] for j in range(1, 10)) == 1)
        for j in range(2, 10):
            model2.addConstr(gp.quicksum(y[j - 1][k - 1] for k in range(1, 10)) == 1)

        model2.optimize()
        
        for j in range(1, 10):
            for k in range(1, 10):
                if y[j - 1][k - 1].x == 1:
                    onPlayer[aGame[g]["date"]][playerName[g][j - 1]]["bat"] = k

        print("===== complete game", g, "- stage 2 =====")
        stage2Obj.append(model2.objVal)

    return onPlayer, playerName, stage2Obj


### Optimization

#### 富邦悍將

In [141]:
aTeam = "富邦悍將"
scheduleList = {}
with open(scheduleRoot + aTeam + ".csv", encoding="utf-8") as csvFile:
    csvReader = csv.DictReader(csvFile)
    for row in csvReader:
        scheduleList[row["日期"]] = row

scheduleByWeek = {}
for week in range(1, 9):
    scheduleByWeek[week] = []
    for date in scheduleList.keys():
        if scheduleList[date]["\ufeff週次"] == str(week):
            scheduleByWeek[week].append({
                "date": datetime.strptime(date, "%m月%d日").date().replace(year=2022),
                "field": "".join(scheduleList[date]["場地"].split()),
                "oppo": "".join(scheduleList[date]["對手"].split()),
                "pitcher": "".join(scheduleList[date]["先發投手"].split())
            })

In [142]:
allResult = {}
for week in range(1, 9):
    aMonth = "Apr"
    aGame = scheduleByWeek[week]
    playerName, onPlayer, stage1Obj, stage1ObjWeight = stage1(aTeam, aGame, aMonth)
    onPlayer, playerName = stageDH(aTeam, aGame, aMonth, playerName, onPlayer)
    onPlayer, playerName, stage2Obj = stage2(aTeam, aGame, onPlayer, playerName)

    index = 0
    for date, dayPlayer in onPlayer.items():
        onPlayerList = [None] * 9
        for playerName, playerData in dayPlayer.items():
            onPlayerList[playerData["bat"] - 1] = playerName + " " + fieldMap[playerData["field"]]
        onPlayerList.extend([stage1Obj, stage1ObjWeight, stage2Obj[index]])
        onPlayer[date] = onPlayerList
        index += 1
    allResult.update(onPlayer)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 152 rows, 128 columns and 384 nonzeros
Model fingerprint: 0xbd08a7c0
Variable types: 0 continuous, 128 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 724.3449268
Presolve removed 152 rows and 128 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 841.637 724.345 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.416365752360e+02, best bound 8.416365752360e+02, gap 0.0000%
===== complete game 0 - stage 1 =====
===== complete game 0 - stage DH =====
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)


In [143]:
weekResult = pd.DataFrame(allResult, index=[1, 2, 3, 4, 5, 6, 7, 8, 9, "stage 1", "stage 1 wAVG", "stage 2"])
weekResult

,2022-04-03,2022-04-04,2022-04-07,2022-04-08,2022-04-09,2022-04-10,2022-04-14,2022-04-15,2022-04-16,2022-04-21,...,2022-05-06,2022-05-09,2022-05-10,2022-05-12,2022-05-13,2022-05-17,2022-05-18,2022-05-20,2022-05-21,2022-05-22
1,范國宸 RF,范國宸 RF,范國宸 RF,范國宸 RF,申皓瑋 CF,范國宸 RF,范國宸 RF,范國宸 RF,范國宸 RF,楊晉豪 3B,...,高國輝 LF,林琨笙 C,林琨笙 C,林琨笙 C,霸帝士 DH,葉竹軒 2B,林琨笙 C,葉竹軒 2B,葉竹軒 2B,葉竹軒 2B
2,申皓瑋 CF,申皓瑋 CF,申皓瑋 CF,申皓瑋 CF,李宗賢 3B,申皓瑋 CF,申皓瑋 CF,申皓瑋 CF,申皓瑋 CF,林哲瑄 DH,...,申皓瑋 CF,申皓瑋 CF,申皓瑋 CF,申皓瑋 CF,申皓瑋 CF,范國宸 1B,申皓瑋 CF,蔣智賢 1B,蔣智賢 1B,蔣智賢 1B
3,李宗賢 3B,李宗賢 3B,李宗賢 3B,李宗賢 3B,王正棠 2B,李宗賢 3B,李宗賢 3B,李宗賢 3B,李宗賢 3B,申皓瑋 CF,...,王正棠 2B,楊瑞承 1B,楊瑞承 1B,楊瑞承 1B,李宗賢 SS,申皓瑋 CF,李宗賢 SS,申皓瑋 CF,申皓瑋 CF,申皓瑋 CF
4,高國麟 LF,高國麟 LF,高國麟 LF,高國麟 LF,林琨笙 C,高國麟 LF,高國麟 LF,高國麟 LF,高國麟 LF,王正棠 2B,...,楊瑞承 1B,葉竹軒 2B,葉竹軒 2B,葉竹軒 2B,葉竹軒 2B,李宗賢 SS,葉竹軒 2B,李宗賢 SS,李宗賢 SS,李宗賢 SS
5,林益全 1B,林益全 1B,林益全 1B,林益全 1B,林益全 1B,林益全 1B,林益全 1B,林益全 1B,林益全 1B,林益全 1B,...,蔣智賢 DH,董子恩 3B,董子恩 3B,董子恩 3B,董子恩 3B,陳真 LF,董子恩 3B,陳真 LF,陳真 LF,陳真 LF
6,張進德 C,張進德 C,張進德 C,張進德 C,張進德 DH,張進德 C,張進德 C,張進德 C,張進德 C,張進德 C,...,范國宸 RF,范國宸 RF,范國宸 RF,范國宸 RF,范國宸 RF,張進德 C,張進德 DH,張進德 C,張進德 C,張進德 C
7,王正棠 2B,王正棠 2B,王正棠 2B,王正棠 2B,高國麟 LF,王正棠 2B,王正棠 2B,王正棠 2B,王正棠 2B,范國宸 RF,...,董子恩 3B,高國輝 LF,高孝儀 LF,高孝儀 LF,高孝儀 LF,王正棠 RF,王正棠 RF,范國宸 RF,范國宸 RF,范國宸 RF
8,高國輝 DH,高國輝 DH,高國輝 DH,高國輝 DH,范國宸 RF,高國輝 DH,高國輝 DH,高國輝 DH,高國輝 DH,高國麟 LF,...,戴培峰 C,蔣智賢 DH,蔣智賢 DH,蔣智賢 DH,戴培峰 C,蔣智賢 DH,陳真 LF,霸帝士 DH,張冠廷 DH,張冠廷 DH
9,于森旭 SS,于森旭 SS,于森旭 SS,于森旭 SS,于森旭 SS,于森旭 SS,于森旭 SS,于森旭 SS,于森旭 SS,于森旭 SS,...,于森旭 SS,于森旭 SS,于森旭 SS,于森旭 SS,楊瑞承 1B,董子恩 3B,范國宸 1B,董子恩 3B,董子恩 3B,董子恩 3B
stage 1,841.637,2971.59,2971.59,2971.59,2971.59,2971.59,2512.28,2512.28,2512.28,2133.34,...,2153.21,2811.93,2811.93,2811.93,2811.93,3529.63,3529.63,3529.63,3529.63,3529.63


In [144]:
weekResult.to_csv("result/" + aTeam + ".csv", encoding='utf_8_sig')

#### 中信兄弟

In [145]:
aTeam = "中信兄弟"
scheduleList = {}
with open(scheduleRoot + aTeam + ".csv", encoding="utf-8") as csvFile:
    csvReader = csv.DictReader(csvFile)
    for row in csvReader:
        scheduleList[row["日期"]] = row

scheduleByWeek = {}
for week in range(1, 10):
    scheduleByWeek[week] = []
    for date in scheduleList.keys():
        if scheduleList[date]["\ufeff週次"] == str(week):
            scheduleByWeek[week].append({
                "date": datetime.strptime(date, "%m月%d日").date().replace(year=2022),
                "field": "".join(scheduleList[date]["場地"].split()),
                "oppo": "".join(scheduleList[date]["對手"].split()),
                "pitcher": "".join(scheduleList[date]["先發投手"].split())
            })

In [146]:
allResult = {}
for week in range(1, 10):
    aMonth = "Apr"
    aGame = scheduleByWeek[week]
    playerName, onPlayer, stage1Obj, stage1ObjWeight = stage1(aTeam, aGame, aMonth)
    onPlayer, playerName = stageDH(aTeam, aGame, aMonth, playerName, onPlayer)
    onPlayer, playerName, stage2Obj = stage2(aTeam, aGame, onPlayer, playerName)

    index = 0
    for date, dayPlayer in onPlayer.items():
        onPlayerList = [None] * 9
        for playerName, playerData in dayPlayer.items():
            onPlayerList[playerData["bat"] - 1] = playerName + " " + fieldMap[playerData["field"]]
        onPlayerList.extend([stage1Obj, stage1ObjWeight, stage2Obj[index]])
        onPlayer[date] = onPlayerList
        index += 1
    allResult.update(onPlayer)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 304 rows, 256 columns and 768 nonzeros
Model fingerprint: 0x844d52c0
Variable types: 0 continuous, 256 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1060.7687758
Presolve removed 289 rows and 235 columns
Presolve time: 0.00s
Presolved: 15 rows, 21 columns, 42 nonzeros
Found heuristic solution: objective 1323.6971745
Variable types: 0 continuous, 21 integer (21 binary)
Found heuristic solution: objective 1323.9802412

Root relaxation: objective 1.326277e+03, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0    

In [147]:
weekResult = pd.DataFrame(allResult, index=[1, 2, 3, 4, 5, 6, 7, 8, 9, "stage 1", "stage 1 wAVG", "stage 2"])
weekResult.to_csv("result/" + aTeam + ".csv", encoding='utf_8_sig')

#### 味全龍

In [148]:
aTeam = "味全龍"
scheduleList = {}
with open(scheduleRoot + aTeam + ".csv", encoding="utf-8") as csvFile:
    csvReader = csv.DictReader(csvFile)
    for row in csvReader:
        scheduleList[row["日期"]] = row

scheduleByWeek = {}
for week in range(1, 8):
    scheduleByWeek[week] = []
    for date in scheduleList.keys():
        if scheduleList[date]["\ufeff週次"] == str(week):
            scheduleByWeek[week].append({
                "date": datetime.strptime(date, "%m月%d日").date().replace(year=2022),
                "field": "".join(scheduleList[date]["場地"].split()),
                "oppo": "".join(scheduleList[date]["對手"].split()),
                "pitcher": "".join(scheduleList[date]["先發投手"].split())
            })

allResult = {}
for week in range(1, 8):
    aMonth = "Apr"
    aGame = scheduleByWeek[week]
    playerName, onPlayer, stage1Obj, stage1ObjWeight = stage1(aTeam, aGame, aMonth)
    onPlayer, playerName = stageDH(aTeam, aGame, aMonth, playerName, onPlayer)
    onPlayer, playerName, stage2Obj = stage2(aTeam, aGame, onPlayer, playerName)

    index = 0
    for date, dayPlayer in onPlayer.items():
        onPlayerList = [None] * 9
        for playerName, playerData in dayPlayer.items():
            onPlayerList[playerData["bat"] - 1] = playerName + " " + fieldMap[playerData["field"]]
        onPlayerList.extend([stage1Obj, stage1ObjWeight, stage2Obj[index]])
        onPlayer[date] = onPlayerList
        index += 1
    allResult.update(onPlayer)

weekResult = pd.DataFrame(allResult, index=[1, 2, 3, 4, 5, 6, 7, 8, 9, "stage 1", "stage 1 wAVG", "stage 2"])
weekResult.to_csv("result/" + aTeam + ".csv", encoding='utf_8_sig')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 143 rows, 120 columns and 360 nonzeros
Model fingerprint: 0x5b637288
Variable types: 0 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 667.8391602
Presolve removed 135 rows and 108 columns
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 24 nonzeros
Found heuristic solution: objective 709.8687121
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 746.1270919

Root relaxation: objective 7.478223e+02, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0         

#### 樂天桃猿

In [149]:
aTeam = "樂天桃猿"
scheduleList = {}
with open(scheduleRoot + aTeam + ".csv", encoding="utf-8") as csvFile:
    csvReader = csv.DictReader(csvFile)
    for row in csvReader:
        scheduleList[row["日期"]] = row

scheduleByWeek = {}
for week in range(1, 9):
    scheduleByWeek[week] = []
    for date in scheduleList.keys():
        if scheduleList[date]["\ufeff週次"] == str(week):
            scheduleByWeek[week].append({
                "date": datetime.strptime(date, "%m月%d日").date().replace(year=2022),
                "field": "".join(scheduleList[date]["場地"].split()),
                "oppo": "".join(scheduleList[date]["對手"].split()),
                "pitcher": "".join(scheduleList[date]["先發投手"].split())
            })

allResult = {}
for week in range(1, 9):
    aMonth = "Apr"
    aGame = scheduleByWeek[week]
    playerName, onPlayer, stage1Obj, stage1ObjWeight = stage1(aTeam, aGame, aMonth)
    onPlayer, playerName = stageDH(aTeam, aGame, aMonth, playerName, onPlayer)
    onPlayer, playerName, stage2Obj = stage2(aTeam, aGame, onPlayer, playerName)

    index = 0
    for date, dayPlayer in onPlayer.items():
        onPlayerList = [None] * 9
        for playerName, playerData in dayPlayer.items():
            onPlayerList[playerData["bat"] - 1] = playerName + " " + fieldMap[playerData["field"]]
        onPlayerList.extend([stage1Obj, stage1ObjWeight, stage2Obj[index]])
        onPlayer[date] = onPlayerList
        index += 1
    allResult.update(onPlayer)

weekResult = pd.DataFrame(allResult, index=[1, 2, 3, 4, 5, 6, 7, 8, 9, "stage 1", "stage 1 wAVG", "stage 2"])
weekResult.to_csv("result/" + aTeam + ".csv", encoding='utf_8_sig')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 709 rows, 568 columns and 1817 nonzeros
Model fingerprint: 0xc2e67b1f
Variable types: 0 continuous, 568 integer (568 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 4130.4605633
Presolve removed 702 rows and 556 columns
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros
Found heuristic solution: objective 4627.0715434
Variable types: 0 continuous, 12 integer (12 binary)

Root relaxation: objective 4.634674e+03, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    4634.6743308 4634.67433  0.00%    

#### 統一7-ELEVEn獅

In [150]:
aTeam = "統一7-ELEVEn獅"
scheduleList = {}
with open(scheduleRoot + aTeam + ".csv", encoding="utf-8") as csvFile:
    csvReader = csv.DictReader(csvFile)
    for row in csvReader:
        scheduleList[row["日期"]] = row

scheduleByWeek = {}
for week in range(1, 9):
    scheduleByWeek[week] = []
    for date in scheduleList.keys():
        if scheduleList[date]["\ufeff週次"] == str(week):
            scheduleByWeek[week].append({
                "date": datetime.strptime(date, "%m月%d日").date().replace(year=2022),
                "field": "".join(scheduleList[date]["場地"].split()),
                "oppo": "".join(scheduleList[date]["對手"].split()),
                "pitcher": "".join(scheduleList[date]["先發投手"].split())
            })

allResult = {}
for week in range(1, 9):
    aMonth = "Apr"
    aGame = scheduleByWeek[week]
    playerName, onPlayer, stage1Obj, stage1ObjWeight = stage1(aTeam, aGame, aMonth)
    onPlayer, playerName = stageDH(aTeam, aGame, aMonth, playerName, onPlayer)
    onPlayer, playerName, stage2Obj = stage2(aTeam, aGame, onPlayer, playerName)

    index = 0
    for date, dayPlayer in onPlayer.items():
        onPlayerList = [None] * 9
        for playerName, playerData in dayPlayer.items():
            onPlayerList[playerData["bat"] - 1] = playerName + " " + fieldMap[playerData["field"]]
        onPlayerList.extend([stage1Obj, stage1ObjWeight, stage2Obj[index]])
        onPlayer[date] = onPlayerList
        index += 1
    allResult.update(onPlayer)

weekResult = pd.DataFrame(allResult, index=[1, 2, 3, 4, 5, 6, 7, 8, 9, "stage 1", "stage 1 wAVG", "stage 2"])
weekResult.to_csv("result/" + aTeam + ".csv", encoding='utf_8_sig')


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 268 rows, 224 columns and 672 nonzeros
Model fingerprint: 0xe16b2762
Variable types: 0 continuous, 224 integer (224 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 939.7817409
Presolve removed 263 rows and 218 columns
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 12 nonzeros
Found heuristic solution: objective 1443.7753191
Variable types: 0 continuous, 6 integer (6 binary)

Root relaxation: objective 1.443937e+03, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1443.9371262 1443.93713  0.00%     -   